In [1]:
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
import pandas as pd
import numpy as np
np.random.seed(0)
from sklearn.model_selection import GridSearchCV, StratifiedKFold  # , cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, precision_recall_fscore_support, confusion_matrix  # , classification_report



In [2]:
DATA_PATH = "./data/"
FEATURES_4D = DATA_PATH + "4dFeatures/"

In [3]:
sn_train = pd.read_pickle(FEATURES_4D + 'allSNS.pickle')

sn_train.shape

(5000, 126)

In [4]:
sn_test = pd.read_pickle(FEATURES_4D + "allMSN.pickle")

sn_test.shape


(153, 126)

In [5]:
allnsn_small = pd.read_pickle(FEATURES_4D+"allNSNsmall.pickle")

allnsn_small.shape

(5500, 126)

In [6]:
nsns_test = allnsn_small.sample(300)

nsns_train = allnsn_small[~allnsn_small.ID.isin(nsns_test.ID)].sample(5000)

nsns_test.shape

(300, 126)

In [7]:
all_train = pd.concat([sn_train,nsns_train])

all_test = pd.concat([sn_test,nsns_test])

all_train.shape

(10000, 126)

In [8]:
all_train.head()

,ID,ObsCount_i,amplitude_i,beyond1st_i,flux_percentile_ratio_mid20_i,flux_percentile_ratio_mid35_i,flux_percentile_ratio_mid50_i,flux_percentile_ratio_mid65_i,flux_percentile_ratio_mid80_i,kurtosis_i,...,poly4_t1_g,poly4_t2_g,poly4_t3_g,poly4_t4_g,skew_g,small_kurtosis_g,std_g,stetson_j_g,stetson_k_g,Class
0,SN_0,21,78.920400,0.285714,0.226183,0.535467,0.793241,0.931034,0.970109,-1.466621,...,-0.177729,0.031785,-0.000283,-0.000008,1.442733,-4.033333,25.138730,16.242553,0.927641,sn
1,SN_1,7,21.993055,0.285714,0.422545,0.709212,0.753952,0.798691,0.915855,-2.377776,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,sn
2,SN_2,5,21.529500,0.400000,0.319354,0.558869,0.798384,0.873990,0.949596,-2.438208,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,sn
3,SN_3,13,72.678250,0.307692,0.097085,0.199656,0.363920,0.582721,0.834838,0.099839,...,-0.336197,0.030121,-0.000030,-0.000015,2.159561,-4.900000,9.075783,2.838324,0.756163,sn
4,SN_4,8,65.732650,0.125000,0.494576,0.508182,0.525156,0.545836,0.760223,0.290385,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,sn


In [9]:
all_test.head()

,ID,ObsCount_i,amplitude_i,beyond1st_i,flux_percentile_ratio_mid20_i,flux_percentile_ratio_mid35_i,flux_percentile_ratio_mid50_i,flux_percentile_ratio_mid65_i,flux_percentile_ratio_mid80_i,kurtosis_i,...,poly4_t1_g,poly4_t2_g,poly4_t3_g,poly4_t4_g,skew_g,small_kurtosis_g,std_g,stetson_j_g,stetson_k_g,Class
0,MSN_10502,21,185.342232,0.095238,0.080170,0.122250,0.175617,0.337986,0.621979,5.160671,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sn
1,MSN_87679,13,34.943539,0.230769,0.303379,0.402537,0.610876,0.872824,0.942403,-1.268704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sn
2,MSN_119183,17,12.828232,0.411765,0.124565,0.183884,0.233649,0.801787,0.920751,-0.219656,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sn
3,MSN_62488,45,59.885384,0.333333,0.081737,0.196849,0.418143,0.578618,0.790034,0.098639,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sn
4,MSN_107646,9,11.143585,0.333333,0.124915,0.312376,0.546724,0.695278,0.862875,-0.790142,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,sn


In [10]:
features = all_train.columns[1:-1]

y_train = pd.factorize(all_train['Class'])[0]
y_train




clf = RandomForestClassifier(n_estimators=50,random_state=0, class_weight='balanced')

Hay nan values....

In [11]:
all_train = all_train.fillna(0)

In [12]:
clf.fit(all_train[features], y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [13]:
all_test = all_test.fillna(0)

In [14]:
preds = clf.predict(all_test[features])

In [15]:
y_test = pd.factorize(all_test["Class"])[0]

In [17]:
pd.crosstab(preds, y_test, rownames=['Predicted '], colnames=['Actual '])

Actual,0,1
Predicted,,
0,59,4
1,94,296
